In [3]:
# installing pyvips for offline use
# intall the deb packages
!yes | dpkg -i --force-depends /kaggle/input/pyvips-python-and-deb-package-gpu/linux_packages/archives/*.deb &> /dev/null
# install the python wrapper
!pip install pyvips -f /kaggle/input/pyvips-python-and-deb-package-gpu/python_packages/ --no-index &> /dev/null

yes: standard output: Broken pipe


In [4]:
import pyvips
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv", usecols=["label", "image_id", "is_tma"])

wsi_dir = Path("/kaggle/input/UBC-OCEAN/train_thumbnails")
tma_dir = Path("/kaggle/input/UBC-OCEAN/train_images")

df["image_id"] = [tma_dir / f"{row['image_id']}.png" if row["is_tma"] else wsi_dir / f"{row['image_id']}_thumbnail.png" for _, row in df.iterrows()]

classes = ['HGSC', 'LGSC', 'EC', 'CC', 'MC']
class2idx = {label: idx for idx, label in enumerate(classes)}

df["label"] = df["label"].map(class2idx)
df["type"] = ["train"] * len(df)
df.rename(columns={"image_id": "slide_id"}, inplace=True)

# In-place editing
df_test = df.sample(frac=0.2)
df.loc[df_test.index, "type"] = ["valid"] * len(df_test)

df = df.drop(columns=["is_tma"])

df.to_csv("wsi_train.csv", index=False)

In [6]:
df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
df.describe()

,image_id,image_width,image_height
count,538.000000,538.000000,538.000000
mean,32194.340149,48859.533457,29729.460967
std,18774.950592,20040.989927,10762.899796
min,4.000000,2964.000000,2964.000000
25%,15881.250000,34509.000000,22089.500000
50%,32152.000000,48160.000000,29732.000000
75%,47892.500000,64143.750000,37880.750000
max,65533.000000,105763.000000,50155.000000


In [7]:
!head wsi_train.csv

slide_id,label,type
/kaggle/input/UBC-OCEAN/train_thumbnails/4_thumbnail.png,0,train
/kaggle/input/UBC-OCEAN/train_thumbnails/66_thumbnail.png,1,train
/kaggle/input/UBC-OCEAN/train_images/91.png,0,train
/kaggle/input/UBC-OCEAN/train_thumbnails/281_thumbnail.png,1,train
/kaggle/input/UBC-OCEAN/train_thumbnails/286_thumbnail.png,2,valid
/kaggle/input/UBC-OCEAN/train_thumbnails/431_thumbnail.png,0,train
/kaggle/input/UBC-OCEAN/train_thumbnails/706_thumbnail.png,0,train
/kaggle/input/UBC-OCEAN/train_thumbnails/970_thumbnail.png,0,train
/kaggle/input/UBC-OCEAN/train_thumbnails/1020_thumbnail.png,0,train


In [8]:
!mkdir output

In [9]:
!rm local_learning_wsi -rf
!git clone https://github.com/karl-joan/local_learning_wsi.git

Cloning into 'local_learning_wsi'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 100 (delta 54), reused 63 (delta 28), pack-reused 0
Receiving objects: 100% (100/100), 442.50 KiB | 12.29 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [10]:
!echo -e "machine api.wandb.ai\n  login user\n  password 33b28a5d3e362bc21dfe1fc1759af32fdd74dec0" >> /root/.netrc

In [11]:
!python local_learning_wsi/main.py /kaggle/input/UBC-OCEAN/train_thumbnails \
    /kaggle/working/wsi_train.csv \
    --num-workers 1 \
    --num-classes 5 \
    --output-dir /kaggle/working/output \
    --precision 16-mixed \
    --gpu-id 0 \
    --batch-size 1 \
    --project-name "UBC Ovarian Cancer Classification" \
    --epochs 5 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a59

In [ ]:
img = pyvips.Image.new_from_file("/kaggle/input/UBC-OCEAN/train_thumbnails/10077_thumbnail.png")

In [20]:
img.numpy().shape

(2210, 3000, 3)

In [21]:
import cv2

img = cv2.cvtColor(cv2.imread("/kaggle/input/UBC-OCEAN/train_thumbnails/10077_thumbnail.png"), cv2.COLOR_BGR2RGB)
img.shape

(2210, 3000, 3)

In [14]:
df.sort_values(by=["image_height"])

,image_id,label,image_width,image_height,is_tma
112,13568,LGSC,2964,2964,True
37,4134,MC,2964,2964,True
355,41586,CC,2964,2964,True
423,50932,HGSC,2964,2964,True
361,42857,CC,2964,2964,True
...,...,...,...,...,...
226,28028,HGSC,48570,49215,False
373,44530,MC,53239,49293,False
256,30986,MC,66911,49395,False
198,24563,CC,60533,49543,False


In [26]:
df = pd.read_csv("wsi_train.csv")

In [28]:
df = df[df["type"] == "valid"]

In [32]:
df.iloc[3]["slide_id"]

'/kaggle/input/UBC-OCEAN/train_thumbnails/1660_thumbnail.png'

In [33]:
cv2.imread("/kaggle/input/UBC-OCEAN/train_thumbnails/1660_thumbnail.png").shape

(736, 3000, 3)

In [16]:
for img_name in Path("/kaggle/input/UBC-OCEAN/train_thumbnails").glob("*.png"):
    shape = pyvips.Image.new_from_file(img_name).numpy().shape
    if any(np.array(shape[:2])) <= 128):
        print(shape)

TypeError: '<=' not supported between instances of 'tuple' and 'int'